In [1]:
import numpy as np
import math
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
im = Image.open("TestImage2c.jpg")
print(im.format, im.size, im.mode)

('JPEG', (1008, 756), 'RGB')


In [3]:
col, row = im.size
data = np.zeros((row * col, 3))
pixels = im.load()
print col, row
for i in range(row):
    for j in range(col):
        r, g, b = pixels[j, i]
        data[col * i + j, :] = r, g, b
grayscale = np.zeros(row * col)
for i in range(row):
    for j in range(col):
        grayscale[col * i + j] = data[col * i + j, 0] * 0.3 + data[col * i + j, 1] * 0.59 + data[col * i + j, 2] * 0.11
grayscale.resize((row, col))

1008 756


In [ ]:
im = Image.fromarray(grayscale)
im.show()
# greyscale done

In [4]:
sobel = np.zeros((row, col, 2))
magnitude = np.zeros((row, col))
thita = np.zeros((row, col))
# Gx, Gy, G, thita
maxGrey = 0
minGrey = 9999

threshold = 20
for i in range(1, row - 1):
    for j in range(1, col - 1):
        sobel[i, j, 0] = grayscale[i - 1, j + 1] + 2 * grayscale[i, j + 1] + grayscale[i + 1, j + 1] \
                         - grayscale[i - 1, j - 1] - 2 * grayscale[i, j - 1] - grayscale[i + 1, j - 1]
        sobel[i, j, 1] = grayscale[i - 1, j - 1] + 2 * grayscale[i - 1, j] + grayscale[i - 1, j + 1] \
                         - grayscale[i + 1, j - 1] - 2 * grayscale[i + 1, j] - grayscale[i + 1, j + 1]
        magnitude[i, j] = (sobel[i, j, 0] ** 2 + sobel[i, j, 1] ** 2) ** 0.5
        if sobel[i, j, 0] == 0:
            thita[i, j] = math.atan(sobel[i, j, 1] / 0.01)
        else:
            thita[i, j] = math.atan(sobel[i, j, 1] / sobel[i, j, 0])
        if magnitude[i, j] > maxGrey:
            maxGrey = magnitude[i, j]
        if magnitude[i, j] < min:
            minGrey = magnitude[i, j]
print maxGrey, minGrey
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        magnitude[i, j] = magnitude[i, j] / maxGrey * 255
        if magnitude[i, j] > threshold:
            magnitude[i, j] = 255
            count += 1
        else:
            magnitude[i, j] = 0
print count

952.571879598 4.01943669423e-14
32060


In [ ]:
im = Image.fromarray(magnitude)
im.show()
# greyscale done

In [5]:
expand_old = np.copy(magnitude)
expand_new = np.copy(magnitude)
flag = 0
for k in range(3):
    for i in range(1, row - 1):
        for j in range(1, col - 1):
            flag = 0
            if expand_old[i-1,j]>0: flag += 1
            if expand_old[i+1,j]>0: flag += 1
            if expand_old[i,j-1]>0: flag += 1
            if expand_old[i,j+1]>0: flag += 1
            if flag>= 2:
                expand_new[i,j] = 255
    expand_old = np.copy(expand_new)

In [ ]:
im = Image.fromarray(expand_new)
im.show()

In [6]:
distance0 = np.copy(expand_new)
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j]>0:
            distance0[i,j]=1
distance_new = np.copy(distance0)
distance_old = np.copy(distance0)
for k in range(15):
    for i in range(1,row-1):
        for j in range(1,col-1):
            min=999
            if distance_old[i-1,j]<min:
                min = distance_old[i-1,j]
            if distance_old[i+1,j]<min:
                min = distance_old[i+1,j]
            if distance_old[i,j-1]<min:
                min = distance_old[i,j-1]
            if distance_old[i,j+1]<min:
                min = distance_old[i,j+1]
            distance_new[i,j]=distance0[i,j]+min
    distance_old = np.copy(distance_new)

for i in range(1,row-1):
    for j in range(1,col-1):
        if distance_old[i,j]<distance_old[i-1,j] or distance_old[i,j]<distance_old[i+1,j] or \
        distance_old[i,j]<distance_old[i,j-1] or distance_old[i,j]<distance_old[i,j+1]:
            distance0[i,j] = 0

count = 0
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j] > 0:
            distance0[i,j]=255
            count +=1

In [ ]:
im = Image.fromarray(distance0)
im.show()
# greyscale done

In [7]:
point = np.zeros((count, 2))
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        if distance0[i, j] == 255:
            point[count, :] = i, j
            count += 1
count

9826

In [8]:
# hough transform
angle = 720
p = 4000
hough = np.zeros((angle, p))
for i in range(count):
    if (i % 1000)==0: print i
    for j in range(angle):
        x, y = point[i,:]
        tmpAngle = j / 720.0 * math.pi
        tmpP = x * math.cos(tmpAngle) + y * math.sin(tmpAngle)
        tmpP = int(tmpP) + 2000
        hough[j, tmpP] +=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [9]:
threshold = 50
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            count += 1
print count

tmpHough = np.zeros((count, 3))
tmpCount = 0
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            tmpHough[tmpCount,:]=hough[i,j],i,j
            tmpCount += 1

print tmpCount
effectiveHough = np.zeros((count, 3))
count = 0
for i in range(tmpCount):
    tmp = tmpHough[i,0]
    tmpAngle=tmpHough[i,1]
    tmpP=tmpHough[i,2]
    top = 0 if tmpAngle - 10 < 0 else int(tmpAngle - 10)
    bottom = angle - 1 if tmpAngle + 10 > angle - 1 else int(tmpAngle + 10)
    left = 0 if tmpP - 20 < 0 else int(tmpP - 20)
    right = p - 1 if tmpP + 20 > p - 1 else int(tmpP + 20)
    maxFlag = 1       
    for k in range(top, bottom + 1):
            for l in range(left, right + 1):
                if tmp < hough[k, l]:
                    maxFlag = 0
    if maxFlag == 1:
        effectiveHough[count, :] = tmp, tmpAngle, tmpP
        count+=1
print count
    
# for i in range(0, angle):
#     for j in range(0, p):
#         tmp = hough[i, j]
#         top = 0 if i - 3 < 0 else i - 3
#         bottom = angle - 1 if i + 3 > angle - 1 else i + 3
#         left = 0 if j - 10 < 0 else j - 10
#         right = p - 1 if j + 10 > p - 1 else j + 10
#         maxFlag = 1
#         for k in range(top, bottom + 1):
#             for l in range(left, right + 1):
#                 if tmp < hough[k, l]:
#                     maxFlag = 0
#         if (tmp > threshold) and maxFlag == 1:
#             effectiveHough[count, :] = hough[i, j], i, j
#             count += 1
# print count
#get effectiveHough

# for i in range(100):
#     print effectiveHough[i,0]

for i in range(count):
    for j in range(i + 1, count):
        if effectiveHough[i, 0] < effectiveHough[j, 0]:
            tmpNum, tmpAngle, tmpP = effectiveHough[i, :]
            effectiveHough[i] = effectiveHough[j]
            effectiveHough[j] = tmpNum, tmpAngle, tmpP

#sort effectiveHough

11712
1886
81


In [10]:
for i in range(100):
    print effectiveHough[i]

[  241.   142.  2837.]
[  237.   645.  1576.]
[  224.   646.  1673.]
[  218.   147.  2902.]
[  207.   167.  2330.]
[  207.   221.  2711.]
[  185.   656.  1903.]
[  168.   692.  1815.]
[  142.   686.  1760.]
[  109.   692.  1738.]
[  107.   214.  2659.]
[  102.    35.  2020.]
[   99.   351.  2468.]
[   86.    18.  2401.]
[   86.   715.  1642.]
[   86.   716.  1642.]
[   85.   396.  2018.]
[   85.   397.  2016.]
[   76.    19.  2356.]
[   76.    79.  2412.]
[   75.   700.  1703.]
[   73.   139.  2454.]
[   73.     4.  2313.]
[   72.   525.  2000.]
[   70.   713.  1694.]
[   69.   687.  1672.]
[   68.   340.  2634.]
[   67.   605.  2000.]
[   67.   324.  2531.]
[   67.    98.  2427.]
[   66.    47.  2350.]
[   65.   671.  1823.]
[   65.   673.  1696.]
[   64.   576.  1832.]
[   64.    51.  2417.]
[   64.    73.  2446.]
[   63.   560.  2000.]
[   63.   222.  2515.]
[   62.    34.  2335.]
[   61.   649.  2000.]
[   61.   303.  2814.]
[   61.   233.  2494.]
[   60.   589.  1783.]
[   60.   2

In [11]:
resultImage = np.copy(magnitude)
for i in range(row):
    for j in range(col):
        for k in range(20):
            if int(i * math.cos(effectiveHough[k, 1] / 720.0 * math.pi) +
                                   j * math.sin(effectiveHough[k, 1] / 720.0 * math.pi)) - effectiveHough[
                k, 2] +2000 == 0:
                resultImage[i, j] = 255

In [24]:
im = Image.fromarray(resultImage)
im.show()

In [25]:
im.save("houghtansform.jpg","JPEG")

IOError: cannot write mode F as JPEG

### effectiveHough 数组中的内容：
* 落在该直线上点的数量
* thita:角度，取值0-720
* p:直线到原点距离，取值-2000-2000

#### 该数组已sort

### 直线——线段
* 对图中所有点搜索
* effectiveLinePoint[100,max(effectiveHough[0]),2],如果点在前一百的hough直线上，加入到数组中
* 对数组作sort
* 求出数组中的线段

In [12]:
maxEffectvieHough = int(effectiveHough[0,0])
effectiveLinePoint = np.zeros((100,500,2))
lineCount = np.zeros(100)
for i in range(row):
    for j in range(col):
        if magnitude[i,j]==255:
            for k in range(100):
                onLine = int(i * math.cos(effectiveHough[k, 1] / 720.0 * math.pi) + 
                    j * math.sin(effectiveHough[k, 1] / 720.0 * math.pi)) - effectiveHough[k, 2] +2000 
                if onLine==0:
                    tmp = int(lineCount[k])
                    effectiveLinePoint[k,tmp,:]=i,j
                    lineCount[k]+=1
for i in range (20):
    print lineCount[i]

305.0
328.0
397.0
309.0
248.0
260.0
246.0
257.0
224.0
263.0
170.0
127.0
168.0
155.0
211.0
211.0
111.0
111.0
155.0
139.0


In [13]:
for i in range(100):
    count = int(lineCount[i])
    for j in range(count):
        for k in range(j+1,count):
            if effectiveLinePoint[i, j, 0] > effectiveLinePoint[i, k, 0]:
                tmpI, tmpJ = effectiveLinePoint[i, j, :]
                effectiveLinePoint[i, j, :] = effectiveLinePoint[i, k, :]
                effectiveLinePoint[i, k, :] = tmpI, tmpJ

In [ ]:
effectiveHough[5,:]

In [ ]:
for i in range(int(lineCount[5])):
    print effectiveLinePoint[5,i,:]

### 已有图片

In [14]:
houghLines = np.zeros((row, col))
for i in range(50):
    for j in range(int(lineCount[i])):
        x,y = effectiveLinePoint[i,j,:]
        houghLines[int(x),int(y)]=255

In [20]:
im = Image.fromarray(magnitude)
im.show()

In [ ]:
im = Image.fromarray(distance0)
im.show()

In [ ]:
im = Image.fromarray(resultImage)
im.show()

In [23]:
im = Image.fromarray(houghLines)
im.show()

# 线段化

In [15]:
threshold = 50
# thita,p; start point; end point
effectiveLine = np.zeros((1000,3,2))
length = np.zeros(1000)
pointCount = 0
index = 0
from IPython.core.debugger import Tracer

for i in range(100):
    pointCount = 0
    count = int(lineCount[i])
    startX, startY = effectiveLinePoint[i,0,:]
    oldX, oldY = startX, startY
    for j in range(1,count):
        newX, newY = effectiveLinePoint[i,j,:]
        distance = math.sqrt((oldX-newX) **2 + (oldY-newY) **2 )
        if distance<15:
            oldX,oldY = newX, newY
            pointCount += 1
            if j==count-1:
                #Tracer()()
                distance = math.sqrt((oldX-startX) **2 + (oldY-startY) **2 )
                if distance >= 60:
                    effectiveLine[index,0,:]=effectiveHough[i,1], effectiveHough[i,2]
                    effectiveLine[index,1,:]=startX,startY
                    effectiveLine[index,2,:]=oldX,oldY
                    length[index] = distance
                    index += 1
        else:
            #Tracer()()
            distance = math.sqrt((oldX-startX) **2 + (oldY-startY) **2)
            if distance >= 60:
                effectiveLine[index,0,:]=effectiveHough[i,1], effectiveHough[i,2]
                effectiveLine[index,1,:]=startX,startY
                effectiveLine[index,2,:]=oldX,oldY
                length[index]=distance
                index += 1
            startX, startY = newX, newY
            oldX, oldY = startX, startY
            pointCount = 0
print index

62


In [21]:
for i in range(index):
    print i,effectiveLine[i,:]

0 [[  142.  2837.]
 [  446.   817.]
 [  621.   572.]]
1 [[  645.  1576.]
 [  480.    94.]
 [  602.   452.]]
2 [[  646.  1673.]
 [  372.    79.]
 [  497.   454.]]
3 [[  147.  2902.]
 [  477.   869.]
 [  662.   622.]]
4 [[  167.  2330.]
 [  184.   290.]
 [  319.   139.]]
5 [[  221.  2711.]
 [  367.   611.]
 [  603.   448.]]
6 [[  656.  1903.]
 [  185.   290.]
 [  256.   540.]]
7 [[  692.  1815.]
 [  282.   771.]
 [  308.   984.]]
8 [[  686.  1760.]
 [  356.   755.]
 [  389.   977.]]
9 [[  692.  1738.]
 [  317.   425.]
 [  324.   485.]]
10 [[  692.  1738.]
 [  357.   755.]
 [  385.   982.]]
11 [[  214.  2659.]
 [  279.   614.]
 [  454.   485.]]
12 [[  3.50000000e+01   2.02000000e+03]
 [  1.00000000e+00   1.25000000e+02]
 [  2.10000000e+01   1.00000000e+00]]
13 [[  351.  2468.]
 [  493.   449.]
 [  603.   445.]]
14 [[  715.  1642.]
 [  362.   150.]
 [  363.   211.]]
15 [[  715.  1642.]
 [  377.   856.]
 [  380.   983.]]
16 [[  716.  1642.]
 [  373.   827.]
 [  375.   924.]]
17 [[  396.  20

In [22]:
for i in range(index):
    print length[i]

301.081384346
378.216869005
395.284707521
308.600064809
202.548759562
286.818758103
259.886513694
214.580987042
224.439301371
60.4069532422
228.72035327
217.407451574
125.602547745
110.072703247
61.0081961707
127.035428129
97.0206163658
110.317722964
111.462998345
71.5122367151
61.2045749924
84.3800924389
199.812412027
97.0824391947
65.1152823844
85.0529246999
111.449540152
63.1347764707
87.2353139503
173.769962882
62.3698645181
66.6108099335
72.6704891961
91.7060521449
71.309185944
92.9623579735
79.61155695
65.7343137182
67.2011904656
68.7968022513
66.1891229735
62.3698645181
82.4621125124
67.7421582178
81.0246875958
80.4300938704
83.8629834909
60.827625303
65.8559033041
66.6033032214
63.5609943283
60.5392434707
76.2758153021
63.2850693292
61.0573500899
67.082039325
96.8762096699
60.8029604542
73.2393336944
76.9025357189
74.6257328272
60.207972894


In [16]:
lines = np.zeros((row, col))
for i in range(index):
    angle, p = effectiveLine[i,0,:]
    startX, startY=effectiveLine[i,1,:]
    endX, endY=effectiveLine[i,2,:]
    startX = int(startX)
    endX = int(endX)
    for j in range(startX+1,endX):
        maxY = int(max(startY,endY))
        minY = int(maxY-abs(startY-endY))
        for k in range(minY,maxY+1):
            onLine = int(j * math.cos(angle / 720.0 * math.pi) + 
                    k * math.sin(angle / 720.0 * math.pi) - p) +2000
            if onLine == 0:
                lines[j,k]=255
    im = Image.fromarray(lines)
    if im.mode != 'RGB':
        im = im.convert('RGB')
    im.save(str(i)+".jpg","JPEG")

In [18]:
im = Image.fromarray(lines)
im.show()

In [17]:
linePair = np.zeros((1000,2))
count = 0
for i in range(index):
    for j in range(i+1,index):
        angleDif=abs(effectiveLine[i,0,0]-effectiveLine[j,0,0])
        lengthDif=abs(length[i]-length[j])/max(length[i],length[j])
        pDif=abs(effectiveLine[i,0,1]-effectiveLine[j,0,1])
        if angleDif<=50 and lengthDif <= 0.4 and pDif>10:
            linePair[count,:]=i,j
            count+=1
print count

176


In [ ]:
lines = np.zeros((row, col))
for pair in range(count):
    for i in linePair[pair,:]:
        i = int(i)
        angle, p = effectiveLine[i,0,:]
        startX, startY=effectiveLine[i,1,:]
        endX, endY=effectiveLine[i,2,:]
        startX = int(startX)
        endX = int(endX)
        for j in range(startX+1,endX):
            for k in range(0,col):
                onLine = int(j * math.cos(angle / 720.0 * math.pi) + 
                    k * math.sin(angle / 720.0 * math.pi) - p) +2000
                if onLine == 0:
                    lines[j,k]=255

In [ ]:
im = Image.fromarray(lines)
if im.mode != 'RGB':
    im = im.convert('RGB')
im